In [5]:
# Importing required libraries
from requests import get
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep, time
from random import randint
from IPython.display import clear_output
import pandas as pd

In [2]:
# Creating lists for storing Data
Symbols = []
Names = []
Categories =[]
Prices = []
DayPercents = []
WeekPrecets = []
MarketCap = []
VDollars = []
VCrypto =[]
CSupply = []
WeekGraph = []

# Preparing the monitoring of loop
start_time = time()
requests = 0

# Base URL
base_url = 'https://coinmarketcap.com/?page='

# Iterating through pages
pages = [str(i) for i in range(1, 52)]
for page in pages:
    browser=webdriver.Chrome()
    browser.get(base_url + page)
    
    # Smooth scorlling of pgae (lazy loading)
    total_height = int(browser.execute_script("return document.body.scrollHeight"))
    for i in range(1, total_height, 5):
        browser.execute_script("window.scrollTo(0, {});".format(i))
    sleep(40) # Waiting will help load all data    
    
    # Pause the loop
    sleep(randint(10,15))
    
    # Monitot the request
    requests += 1
    elapsed_time = time() - start_time
    clear_output(wait = True)
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
     
    # Break the loop if the number of requests is greater than expected
    if requests > 52:
        warn('Number of requests was greater than expected.')
        break       
    
    # Parse the content of the request with BeautifulSoup
    html = browser.page_source
    html_soup = BeautifulSoup(html,'html.parser')

    # Select all the cryptcurrency data
    crypto_collection = html_soup.find('div', class_ = 'tableWrapper___3utdq cmc-table-homepage-wrapper___22rL4')
    crypto_data = crypto_collection.tbody.find_all('tr')
    
    # Extract data for individula row 
    for data in crypto_data:
        
        # Extract Symbol
        sym = data.find_all('td')[2].find('img').attrs['src']
        Symbols.append(sym)
        
        # Extract Name
        name = data.find('div',class_ = 'sc-16r8icm-0 sc-1teo54s-1 lgwUsc').p.text
        Names.append(name)
        
        # Extract Category
        category = data.find('div', class_ = 'sc-1teo54s-2 fZIJcI').p.text
        Categories.append(category)
        
        # Extracting Price
        price = float(data.find_all('td')[3].text.lstrip('$').replace(",",""))
        Prices.append(price)
        
        # Extracting # 24H% variation
        dayVar = data.find_all('td')[4]
        valueFlag1 = 0
        if valueFlag1 == 0:
            if dayVar.find('span', class_ = 'icon-Caret-up') is not None:
                dayVar = data.find_all('td')[4].find_all('span')[0].text  + str(" %Up")
            else:
                dayVar =  data.find_all('td')[4].find_all('span')[0].text + str(" %Down")
            if dayVar is not None:
                valueFlag1 = 1
            else:
                sleep(1)
        DayPercents.append(dayVar)    
        
        # Extracting 7d %
        week = data.find_all('td')[5] 
        valueFlag2 = 0
        if valueFlag2 == 0:
            if week.find('span', class_ = 'icon-Caret-up') is not None:
                week = data.find_all('td')[5].find_all('span')[0].text  + str(" %Up")
            else:
                week =  data.find_all('td')[5].find_all('span')[0].text + str(" %Down")
            if week is not None:
                valueFlag2 =1
            else:
                sleep(1)
        WeekPrecets.append(week)
        
        # Extracting Market Cap
        mcap = float(data.find_all('td')[6].text.lstrip('$').replace(",",""))
        MarketCap.append(mcap)
            
        # Extracting Volume 24h in $
        vDollars = float(data.find_all('td')[7].find_all('p')[0].text.lstrip('$').replace(",",""))
        VDollars.append(vDollars)    
        
        # Extracitng  Volume 24h in crypto
        vCrypto = float(data.find_all('td')[7].find_all('p')[1].text.lstrip('$').replace(category,"").replace(",","")) 
        VCrypto.append(vCrypto)
        
        # Extracting circulaitng Supply
        csupply = float(data.find_all('td')[8].p.text.rstrip(category).replace(",",""))
        CSupply.append(csupply)
        
        # Extracing last 7 day Grpah
        graph = data.find_all('td')[9].find('img').attrs['src']
        WeekGraph.append(graph)
    browser.close()
    
# converting to pandas
pd.options.display.float_format = '{:.2f}'.format # avoid values from converting to exponent form
df = pd.DataFrame({
    "Symbols":Symbols,
    "Name":Names,
    "Categorie":Categories,
    "Price":Prices,
    "24h %":DayPercents,
    "7d %":WeekPrecets,
    "Market Cap":MarketCap,
    "Volume (24H) in  $":VDollars,
    "Volume (24H) in  $":VCrypto,
    "Circulating Supply":CSupply,
    "Week Changes Graph":WeekGraph
})       
print(df.info())
df    

Request: 1; Frequency: 0.00887370926507368 requests/s
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Symbols             100 non-null    object 
 1   Name                100 non-null    object 
 2   Categorie           100 non-null    object 
 3   Price               100 non-null    float64
 4   24h %               100 non-null    object 
 5   7d %                100 non-null    object 
 6   Market Cap          100 non-null    float64
 7   Volume (24H) in  $  100 non-null    float64
 8   Circulating Supply  100 non-null    float64
 9   Week Changes Graph  100 non-null    object 
dtypes: float64(4), object(6)
memory usage: 5.5+ KB
None


,Symbols,Name,Categorie,Price,24h %,7d %,Market Cap,Volume (24H) in $,Circulating Supply,Week Changes Graph
0,https://s2.coinmarketcap.com/static/img/coins/...,Bitcoin,BTC,49205.39,1.29% %Up,15.15% %Down,920700722918.00,1191067.00,18711381.00,https://s3.coinmarketcap.com/generated/sparkli...
1,https://s2.coinmarketcap.com/static/img/coins/...,Ethereum,ETH,3818.81,1.89% %Down,2.37% %Down,442580133422.00,10845234.00,115894820.00,https://s3.coinmarketcap.com/generated/sparkli...
2,https://s2.coinmarketcap.com/static/img/coins/...,Binance Coin,BNB,599.41,2.61% %Up,11.42% %Down,91968725491.00,6008762.00,153432897.00,https://s3.coinmarketcap.com/generated/sparkli...
3,https://s2.coinmarketcap.com/static/img/coins/...,Cardano,ADA,2.38,9.43% %Up,37.69% %Up,76039052604.00,6544327154.00,31948309441.00,https://s3.coinmarketcap.com/generated/sparkli...
4,https://s2.coinmarketcap.com/static/img/coins/...,Dogecoin,DOGE,0.53,1.97% %Up,6.25% %Up,68708963006.00,17438062282.00,129624091405.00,https://s3.coinmarketcap.com/generated/sparkli...
...,...,...,...,...,...,...,...,...,...,...
95,https://s2.coinmarketcap.com/static/img/coins/...,SwissBorg,CHSB,1.13,0.54% %Down,15.02% %Down,1126035770.00,4052007.00,1000000000.00,https://s3.coinmarketcap.com/generated/sparkli...
96,https://s2.coinmarketcap.com/static/img/coins/...,TrueUSD,TUSD,1.00,0.01% %Up,0.04% %Up,1125077235.00,330255249.00,1125523285.00,https://s3.coinmarketcap.com/generated/sparkli...
97,https://s2.coinmarketcap.com/static/img/coins/...,Celo,CELO,4.71,0.75% %Down,11.40% %Down,1058973043.00,4700858.00,225479897.00,https://s3.coinmarketcap.com/generated/sparkli...
98,https://s2.coinmarketcap.com/static/img/coins/...,Ankr,ANKR,0.14,3.74% %Up,9.09% %Down,1006957172.00,666211182.00,6996232711.00,https://s3.coinmarketcap.com/generated/sparkli...
